# 实现Lenet-5模型对CIFAR-10数据集进行分类

## LenNet-5模型

LenNet-5模型是Yan Lecun在1998年提出的一种卷积神经网络模型，最初设计用于手写数字的识别，是早期卷积神经网络中最有代表性的实验系统之一。[论文原地址链接](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf)。以下是该模型的结构。![lennet](https://img-blog.csdn.net/20171018154341615?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvaGFwcHlvcmc=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

## CIFAR-10数据集

CIOWE10数据集包括60000个32×32彩色图像，分为10个类别，每个类有6000个图像。有50000个训练图像和10000个测试图像。
数据集分为五个训练批次和一个测试批次，每一个批次都有10000个图像。测试批次包含来自每个类别的随机抽取的1000个图像。训练批次在剩余图像中随机抽取，训练批次可能包含某一类的图像多于另一类。训练批次包含来自每个类的5000个图像。
下面是数据集中的类别以及10个随机抽取的图像：
![](https://www.cs.toronto.edu/~kriz/cifar-10-sample/frog8.png![image.png](attachment:image.png))